In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import os

file_path = '/workspaces/Machine-Learning-Model/Recommendation-system/main.ipynb'
file_location = os.path.dirname(file_path)

print('File location:', file_location)
# BEGIN: Check other files in the folder
folder_path = os.path.dirname(file_path)
files_in_folder = os.listdir(folder_path)

files_containing_file = [file for file in files_in_folder if file_path in file]

print('Files containing the specified file:')
for file in files_containing_file:
    print(file)
# END: Check other files in the folder


File location: /workspaces/Machine-Learning-Model/Recommendation-system
Files containing the specified file:


In [64]:

u_cols = ['user', 'age', 'sex, occupation', 'zip_code']
users = pd.read_csv('/workspaces/Machine-Learning-Model/Recommendation-system/Data/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
n_users = users.shape[0]
print('Number of users:', n_users)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('/workspaces/Machine-Learning-Model/Recommendation-system/Data/ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('/workspaces/Machine-Learning-Model/Recommendation-system/Data/ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

Number of users: 943
Number of traing rates: 90570
Number of test rates: 9430


In [90]:
print(rate_test[:5,:10])

[[       -6        13         4 887431883]
 [       -6        26         4 878542699]
 [       -6        54         4 878542420]
 [       -6       110         3 874965739]
 [       -6       148         2 878542201]]


In [4]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure','Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('/workspaces/Machine-Learning-Model/Recommendation-system/Data/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

X0 = items.values
X_train_counts = X0[:, -19:]

n_items = items.shape[0]
print('Number of items:', n_items)



Number of items: 1682


In [5]:
print(rate_train[:5, :])

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 [        1         5         3 889751712]]


In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm='l2')
X = transformer.fit_transform(X_train_counts.tolist()).toarray()
print(X[:5, :])

[[0.         0.         0.         0.74066017 0.57387209 0.34941857
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.53676706 0.65097024 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.53676706 0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.         0.
  0.        ]
 [0.         0.71065158 0.         0.         0.         0.5397592
  0.         0.         0.45125862 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.735504   0.         0.36318585 0.         0.         0.
  0.         0.         0.         0.         0.57195272 0.
  0.        ]]


In [7]:
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0]
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)



In [8]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
d = X.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros(n_users)
for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    model = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = X[ids, :]
    model.fit(Xhat, scores)
    W[:, n] = model.coef_
    b[n] = model.intercept_


In [9]:
Yhat = X.dot(W) + b

In [10]:
n = 10
np.set_printoptions(precision=2) # 2 digits after .
ids, scores = get_items_rated_by_user(rate_test, n)
print('Rate test:', ids)
print('Scores:', scores)
print('Predictions:', Yhat[ids, n])
print('Error:', np.sqrt(np.mean((Yhat[ids, n] - scores)**2)))

Rate test: [189 229 311 516 560 659 713 719 740 745]
Scores: [3 4 4 2 2 3 4 1 5 4]
Predictions: [3.47 3.13 4.15 4.16 3.9  4.01 3.83 3.56 3.14 3.31]
Error: 1.4449976697815012


In [11]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred
        se += (e*e).sum(axis = 0)
        cnt += e.size
    return np.sqrt(se/cnt)
print('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.9073946349009766
RMSE for test    : 1.2532350678563369


In [87]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from __future__ import print_function 
import pandas as pd 
import numpy as np


class CF(object):
    """docstring for CF"""
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k # number of neighbor points
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
    def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:] 
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalize)
        return self.__pred(i, u, normalize)
    def recommend(self, u, normalized = 1):
        """
        Determine all items should be recommended for user u. (uuCF =1)
        or all users who might have interest on item u (uuCF = 0)
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 
            

In [88]:
rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()
# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [89]:
rs = CF(rate_train, k = 30, uuCF = 1)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print('User-user CF, RMSE =', RMSE)

ValueError: negative row index found